<a href="https://colab.research.google.com/github/nicas83/unipi-aif/blob/main/GNicassioReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Configuration (Linux)

In [ ]:
!sudo apt-get update
!sudo apt-get install python3.10

!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.10 1
!sudo update-alternatives --config python

In [ ]:
!apt install python3.10-venv
!python3.10 -m venv py310_env

!source py310_env/bin/activate

!py310_env/bin/python -m pip install ipykernel
!py310_env/bin/python -m ipykernel install --user --name=py310_env --display-name "Python 3.10 (venv)"

In [ ]:
!py310_env/bin/python --version

In [ ]:
!apt-get update
!apt-get install -y build-essential swig
!apt-get install -y python3.10-dev python3.10 python3.10-distutils
!apt-get install -y libgl1-mesa-glx
!apt-get install -y \
    autoconf \
    libtool \
    pkg-config \
    git \
    flex \
    bison \
    libbz2-dev \
    libzmq3-dev \
    cmake \
    ninja-build \
    libgl1-mesa-glx

In [ ]:
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.10 get-pip.py

!python3.10 -m pip install \
    numpy \
    pybind11 \
    'gym==0.23' \
    setuptools \
    wheel

In [ ]:
!git clone --recursive https://github.com/facebookresearch/nle
#!cd nle && python3.10 -m pip install -e .
!cd nle && VERBOSE=1 python3.10 -m pip install -e .

#!git clone https://github.com/facebookresearch/minihack.git
#!cd minihack && python3.10 -m pip install -e .

!python3.10 -m pip install minihack

In [ ]:
Test
import sys
print(f"Versione Python in uso: {sys.version}")
print(f"Percorso Python: {sys.executable}")

!python3.10 -c "import site; print(site.getsitepackages())"

import site
import sys
sys.path.append('/usr/local/lib/python3.10/dist-packages')

try:
    import minihack
    print("minihack importato correttamente")
except ImportError as e:
    print(f"Errore nell'importazione: {e}")

# Code Import for GitHub

In [ ]:
Import the code from the GitHub below
!git clone https://github.com/nicas83/unipi-aif.git

import sys
import os

sys.path.append('/content/unipi-aif')

# Artificial Intelligence Fundamentals 2024/2025
# Smart Pathfinder and Strategy Comparison for NetHack agent

### Group Name: StocassioAIF

### Member: Gaetano Nicassio

#### Introduction
The project is based on the MiniHack-Skill-Custom-v0 environment and uses the LevelGenerator to create a more complex environment within which the agent will have to move. It is about finding an optimal path using a few steps from the starting point to the target point, while escaping the danger of crossing the path with the wolf (the monster) and navigating around the lava cell (obstacles).
This project has a dual purpose. On the one hand, we calculate the agent's moves based on the best possible path, through the use of an informed search algorithm based on a heuristic and information that defines the distance from the goal. On the other hand, we considered the minihack environment as a competitive game in which the best move made by the agent also depends on what it observes of the environment and the other agents (the wolf in the case of minihack).
To do this, two algorithms were used, **A*-search** for the “dynamic” pathfinding and **MinMax** for the competitive environment.


#### Related Works
In this project, I used the Nethack learning environment and Minihack. This type of environment is a Reinforcement Learning environment commonly used to provide a standard interface to the game. The gym interface engraved in this environment allows it to be manipulated for the development of the artificial agent. Hence, we were able to use Minihack to design a unique environment filled with challenging tasks and obstacles. It allows the creation of distinct settings and provides useful tools.


#### Methodologies and Strategies
The main theme of the project NLE allowed to use Minihack and Gym. Also, it was written in Python and put together in Jupyter Notebook.

The first algorithm used is the A-star. It is a search algorithm that uses two heuristics: the first is given by the cost of the chosen path, the second is a function that, in the strategy used, assigns a score to the path based on the distance from the target (positive score) and from the monster (negative score). The higher the score of h, the better the chosen path will be.

In this experiment we made the A*-search strategy reactive with respect to the monster's moves. Once the optimal path to the target has been identified, the agent's move is executed (I perform a move from the current state S to the first state S' of the calculated path) and following the monster's move, the optimal path is recalculated based on its position.

The second algorithm used is the MinMax-search, an algorithm that operates in competitive environments and that probably adapts better to this type of context. It is a recursive algorithm that alternates the hero's moves (MAX) with the monster's moves (MIN). One of the cons of this algorithm is that the search space could be potentially infinite.
To compensate for the exponential computation time, the alpha-beta pruning technique was used. This technique allows to save computation time by excluding parts of the tree that we already know to be of lower value than what has already been analyzed.

The strategy used to evaluate the path at each move was the same for both algorithms used.
This is based on the distance of the agent from the target and the monster using the Manatthan distance: at each move evaluated, a score is calculated given by the sum of the two distances. The closer we are to the target, the higher the assigned score will be. On the other hand, the closer we are to the monster, the higher the negative score assigned will be, consequently reducing the overall score.
To prevent the agent from moving away from the target when the monster is also close to the target, the penalty of the heuristic based on the distance from the monster is reduced, giving a greater weight to the distance from the target and based on the assumption that the hero dies when bitten 3 times.

Let's start importing the needed library

In [ ]:
import time
from environment import create_level
from moving import DynamicPathfindingAgent
from utils import is_valid_position
from search import Astar_search, minMax_search


In [ ]:

direction_to_action = {
            (-1, 0): 0,  # Moving N
            (0, 1): 1,  # Moving E
            (1, 0): 2,  # Moving S
            (0, -1): 3,  # Moving W
            (-1, 1): 4,  # Moving NE
            (1, 1): 5,  # Moving SE
            (1, -1): 6,  # Moving SW
            (-1, -1): 7,  # Moving NW
            (0, 0): 46  # Rest
        }

def convert_to_action(current, next_pos):
    """Converte una mossa in un indice di azione."""
    dx = next_pos[0] - current[0]
    dy = next_pos[1] - current[1]
    return direction_to_action.get((dx, dy))

Let's define the main function responsible of the agent's behavior

In [ ]:
def run_agent(env, game_map, algorithm='astar'):
    agent = DynamicPathfindingAgent(map_width=11, map_height=11)
    done = False

    try:
        # Visualizza lo stato iniziale
        print(f"Algorithm used to pathfinder: {algorithm}")
        print(f"Initial position: {game_map.get_player_location()}")
        print(f"Target position: {game_map.get_goal_location()}")
        print(f"Monster position: {game_map.get_monsters_location()}")

        paths=[]
        moves=0
        while not done:
            # Visualizza lo stato corrente
            chars_map, hero_pos, monster_pos, goal_pos = game_map.get_map_state()
            if not is_valid_position(hero_pos, chars_map):
                print("ATTENZIONE: Posizione attuale non sicura!")
                break

            # action, path = get_next_move(chars_map, hero_pos, monster_pos, goal_pos, algorithm)
            if algorithm=='astar':
                path = Astar_search(chars_map, hero_pos, monster_pos, goal_pos)
            else:
                path = minMax_search(chars_map, hero_pos, monster_pos, goal_pos, 4)

            if path and len(path) > 1:
                next_pos = path[1]
                action = convert_to_action(path[0], next_pos)
                paths += [path]
                if action is None:
                    action = 46 #agent rest
            print(f"Path choosen: {path}")

            # Esegui l'azione
            state, reward, done, info = env.step(action)
            moves+=1
            env.render()
            # Breve pausa per vedere il rendering
            time.sleep(0.5)
            new_pos = game_map.get_player_location()
            if new_pos == goal_pos:
                print("YOU WIN! CONGRATULATIONS IT'S TIME TO HAVE A 30L")
                break
        return paths, moves
    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()

Applying the A*-search algorithm to the game produced the following results:

In [ ]:
from pprint import pprint

width = 11
height = 11
level = 2
start_pos = (10, 0)
target_pos = (2, 0)
monster_pos = (0,0)
game_map, env = create_level(width,height,level,start_pos,target_pos,monster_pos)
algorithm = "astar"
star_time = time.time()
paths, moves = run_agent(env, game_map, algorithm)
end_time = time.time()
total_time = end_time-star_time
print(f"Paths:")
pprint(paths)
print(f"Number of Moves: {moves}")
print(f"Total time: {total_time} seconds")


Hello Agent, welcome to NetHack!  You are a lawful human Caveman.               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                  d.%.......@                                   
                                  .}.}.}.}.}.                                   
                                  ...........                                   
                                  .}.}.}.}.}.                                   
                                  ...........                                   
                                  .}.}.}.}.}.                                   
                           

While applying the MinMax we get these results (it's necessary to reset the environment before running the game with the new approach):

In [ ]:
#setting the stage
width = 11
height = 11
level = 2
start_pos = (10, 0)
target_pos = (2, 0)
monster_pos = (0,0)

#create a new level in the NetHack-Default0 environment
game_map, env = create_level(width,height,level,start_pos,target_pos,monster_pos)
algorithm = "minmax"

#execute the algorithm
start_time = time.time()
paths, moves = run_agent(env, game_map, algorithm)
end_time = time.time()
total_time = end_time-star_time

print(f"Paths:")
pprint(paths)
print(f"Number of Moves: {moves}")
print(f"Total time: {total_time} seconds")


Hello Agent, welcome to NetHack!  You are a lawful human Caveman.               
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                  d.%.......@                                   
                                  .}.}.}.}.}.                                   
                                  ...........                                   
                                  .}.}.}.}.}.                                   
                                  ...........                                   
                                  .}.}.}.}.}.                                   
                           

### Assessment
At the end of it all, the game shows how a nethack human agent is able to find the goal point from the start point by navigating and avoiding monsters and calculating the best path in a minihack environment.

There were two algorithms used, which were A*-search and MinMax-search using the same strategy based on the manatthan distance from the target and the monster.
In both cases, the aim is to obtain an optimal path for the agent, but while A*-search does not explicitly anticipate the monster's moves but adapts at each step by recalculating the path (effective for avoiding short-term risks and moving towards the target), MinMax considers both the agent's and the monster's moves at multiple turns away (4 in the case of the experiment) evaluating the long-term effects.

With the **A*star-search**, the agent was able to move in 8 steps toward the goal point in 4.2 seconds.

Whereas, with **Min-Max-search**, the agent was able to move in 14 steps toward the goal point in 51.7 seconds

As expected, with the dynamic approach of the A*-search we achieve a target achievement in a smaller number of time and moves compared to the MinMax.

### Conclusion
The dynamic approach of A*-search allows to recalculate the optimal path based on new information (monster move), improving the general capabilities of the agent, but increases the computational cost because the algorithm must be executed at each step, instead of calculating everything at once. Furthermore, it does not contain that predictive component typical of the MinMax algorithm that allows us to anticipate the opponent's moves in the long term.

It may be interesting to change the two approaches to design a hybrid system that combines the best features of both algorithms. For example, it could be interesting to evaluate a combination of A*-search to calculate the basic path and then use a strategy similar to MinMax to refine the moves considering possible dynamics of the opponent.

### GitHub metrics

The repository and metrics can be found at this link,https://github.com/nicas83/unipi-aif/tree/main


## Relationship with the course
This project was based on the theme of the Nethack learning environment used in the course.
It was chosen to use two different algorithms trying to find synergies between them. The search algorithm A*seach was taken from the types of informed search algorithms studied. exploring the use of the cost of risk and the optimal path. The algorithm MinMax search was taken from the set of adversarial search algorithms studied in class.


## References

Minihack New Environment:
Available at: https://minihack.readthedocs.io/en/latest/getting-started/interface.html

Minihack package:
Available at: https://minihack.readthedocs.io/en/latest/api/minihack.html

Nethack:
Available at: https://minihack.readthedocs.io/en/latest/about/nethack.html